In [1]:
import os
# The previous installation attempts resulted in a ModuleNotFoundError related to numpy.
# This suggests a numpy version incompatibility with one of the required libraries (likely paramz).
# We will install a specific version of numpy that is compatible with paramz.
!pip install 'paramz>=0.9.5' 'autograd>=1.3' 'networkx>=2.4' 'numpy==1.9' # Install numpy version known to be compatible with paramz
args = os.getenv("ARGS", "release real").split(" ")
release = args[0]
arch = args[1]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
  Using cached scipy-1.16.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.15.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.14.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scipy-1.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
INFO: pip is still looking at mult

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/MAKING KERNEL PICTURES')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import firedrake
else:
    try:
        import firedrake
    except ImportError:
        if release != "development":
            if arch != "complex":
                !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
            else:
                !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-complex.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        else:
            if arch != "complex":
                !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
            else:
                !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-complex.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        import firedrake

In [11]:
from firedrake import Mesh, FunctionSpace
import os
import numpy as np
import networkx as nx

from manifold_matern.cli import argument_parser
from manifold_matern.manifold_matern_gp import ManifoldMaternGP
from manifold_matern.laplace_eigenvalues import get_eigenpairs

from manifold_matern.utils import export_fun, construct_mesh_graph, \
    convert_to_firedrake_function




def construct_ground_truth(mesh):
    mesh_graph = construct_mesh_graph(mesh)

    geodesics = nx.shortest_path_length(mesh_graph, source=0, weight='weight')

    N = mesh.num_vertices()

    ground_truth = np.zeros((N))
    period = 2*np.pi / 0.3 * 2
    for i in range(N):
        distance = geodesics.get(i)
        if distance is not None:
            ground_truth[i] = 2 * np.sin(distance * period + 0.3)
        else:
            # Handle unreachable vertices, assigning 0.0 as a default value
            ground_truth[i] = 0.0

    return ground_truth


# if __name__ == '__main__':
#     parser = argument_parser()
#     args, unknown_args = parser.parse_known_args() # Unpack the tuple

#     mesh = Mesh('/content/drive/MyDrive/MAKING KERNEL PICTURES/Frog Buddha.msh', dim=3)
#     num_eigenpairs = args.num_eigenpairs # Access num_eigenpairs from the Namespace object

#     print('Constructing ground truth. It may take a while')
#     ground_truth = construct_ground_truth(mesh)

#     mayavi_installed = False
#     if args.mayavi:
#         try:
#             import manifold_matern.plotting
#             mayavi_installed = True
#         except ImportError:
#             import warnings
#             warnings.warn('Mayavi does not seem to be installed.\n'
#                           'No mayavi-based plotting will occur.',
#                           RuntimeWarning)

#     V = FunctionSpace(mesh, "Lagrange", 1)

#     if args.eigenpairs_file is None:
#         print('Getting eigenpairs. It may take a while')
#         eigenpairs = get_eigenpairs(mesh, V, num_eigvals=num_eigenpairs,
#                                     eps_target=1e-10)
#     else:
#         print('Reading eigenpairs from a file')
#         eigenpairs_ = np.load(args.eigenpairs_file)
#         eigenpairs = (eigenpairs_[:, 0],
#                       eigenpairs_[:, 1:])

#     X = [455, 1455, 2455, 3455]
#     Y = ground_truth[X]

#     gp = ManifoldMaternGP(mesh, V, X, Y, eigenpairs)
#     gp.kappa.constrain_bounded(0.01, 5.0)
#     gp.sigma_n.fix(1e-15)
#     gp.sigma_f.constrain_bounded(1., 100000.0)

#     if args.seed is not None:
#         print('Setting seed %d' % args.seed)
#         np.random.seed(args.seed)

#     print('Optimizing MLL. It may take a while')
#     gp.optimize_restarts(num_restarts=5)

#     print('Optimization finished. Proceeding to output')
#     mean, variance = gp.predict()
#     std = np.nan_to_num(np.sqrt(variance))
#     samples = gp.posterior_samples(args.num_samples)

#     output_dir = args.output_dir
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     export_fun(os.path.join(output_dir, '0_groud_truth.pvd'),
#                convert_to_firedrake_function(V, ground_truth))
#     export_fun(os.path.join(output_dir, '0_mean.pvd'),
#                convert_to_firedrake_function(V, mean))
#     export_fun(os.path.join(output_dir, '0_std.pvd'),
#                convert_to_firedrake_function(V, std))

#     for s in range(len(samples)):
#         export_fun(os.path.join(output_dir, '0_sample_%d.pvd' % s),
#                    convert_to_firedrake_function(V, samples[s]))

#     if args.mayavi and mayavi_installed:
#         from manifold_matern.plotting import plot_example
#         vmin, vmax = -4., 4.
#         plot_example(os.path.join(output_dir, '1_ground_truth.png'),
#                      mesh, V, ground_truth, X, vmin, vmax)
#         plot_example(os.path.join(output_dir, '1_mean.png'),
#                      mesh, V, mean, X, vmin, vmax)
#         plot_example(os.path.join(output_dir, '1_std.png'),
#                      mesh, V, std, X, std.min(), std.max(), colormap='viridis')
#         for s in range(len(samples)):
#             plot_example(os.path.join(output_dir, '1_sample_%d.png' % s),
#                          mesh, V, samples[s], X, vmin, vmax)

Constructing ground truth. It may take a while


KeyboardInterrupt: 

In [12]:
if __name__ == '__main__':
    parser = argument_parser()
    args, unknown_args = parser.parse_known_args() # Unpack the tuple

    mesh = Mesh('/content/drive/MyDrive/MAKING KERNEL PICTURES/Frog Buddha.msh', dim=3)
    num_eigenpairs = args.num_eigenpairs # Access num_eigenpairs from the Namespace object

    print('Constructing ground truth. It may take a while')
    ground_truth = construct_ground_truth(mesh)

    mayavi_installed = False
    if args.mayavi:
        try:
            import manifold_matern.plotting
            mayavi_installed = True
        except ImportError:
            import warnings
            warnings.warn('Mayavi does not seem to be installed.\n'
                          'No mayavi-based plotting will occur.',
                          RuntimeWarning)

    V = FunctionSpace(mesh, "Lagrange", 1)

    if args.eigenpairs_file is None:
        print('Getting eigenpairs. It may take a while')
        eigenpairs = get_eigenpairs(mesh, V, num_eigvals=num_eigenpairs,
                                    eps_target=1e-10)
    else:
        print('Reading eigenpairs from a file')
        eigenpairs_ = np.load(args.eigenpairs_file)
        eigenpairs = (eigenpairs_[:, 0],
                      eigenpairs_[:, 1:])

    X = [455, 1455, 2455, 3455]
    Y = ground_truth[X]

    gp = ManifoldMaternGP(mesh, V, X, Y, eigenpairs)
    gp.kappa.constrain_bounded(0.01, 5.0)
    gp.sigma_n.fix(1e-15)
    gp.sigma_f.constrain_bounded(1., 100000.0)

    if args.seed is not None:
        print('Setting seed %d' % args.seed)
        np.random.seed(args.seed)

    print('Optimizing MLL. It may take a while')
    gp.optimize_restarts(num_restarts=5)

    print('Optimization finished. Proceeding to output')
    mean, variance = gp.predict()
    std = np.nan_to_num(np.sqrt(variance))
    samples = gp.posterior_samples(args.num_samples)

    output_dir = args.output_dir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    export_fun(os.path.join(output_dir, '0_groud_truth.pvd'),
               convert_to_firedrake_function(V, ground_truth))
    export_fun(os.path.join(output_dir, '0_mean.pvd'),
               convert_to_firedrake_function(V, mean))
    export_fun(os.path.join(output_dir, '0_std.pvd'),
               convert_to_firedrake_function(V, std))

    for s in range(len(samples)):
        export_fun(os.path.join(output_dir, '0_sample_%d.pvd' % s),
                   convert_to_firedrake_function(V, samples[s]))

    if args.mayavi and mayavi_installed:
        from manifold_matern.plotting import plot_example
        vmin, vmax = -4., 4.
        plot_example(os.path.join(output_dir, '1_ground_truth.png'),
                     mesh, V, ground_truth, X, vmin, vmax)
        plot_example(os.path.join(output_dir, '1_mean.png'),
                     mesh, V, mean, X, vmin, vmax)
        plot_example(os.path.join(output_dir, '1_std.png'),
                     mesh, V, std, X, std.min(), std.max(), colormap='viridis')
        for s in range(len(samples)):
            plot_example(os.path.join(output_dir, '1_sample_%d.png' % s),
                         mesh, V, samples[s], X, vmin, vmax)

Constructing ground truth. It may take a while
Getting eigenpairs. It may take a while


Optimizing MLL. It may take a while
Optimization restart 1/5, f = 43329.33264956861
Optimization restart 2/5, f = 6.628353630304342
Optimization restart 3/5, f = 6.62835363029043
Optimization restart 4/5, f = 6.628353630281045
Optimization restart 5/5, f = 6.6283536302834225
Optimization finished. Proceeding to output


/tmp/ipython-input-364407282.py:51: RuntimeWarning: invalid value encountered in sqrt
  std = np.nan_to_num(np.sqrt(variance))
/usr/local/lib/python3.11/dist-packages/firedrake/_deprecation.py:65: UserWarning: The use of `File` for output is deprecated, please update your code to use `VTKFile` from `firedrake.output`.
  warn(
/usr/local/lib/python3.11/dist-packages/firedrake/_deprecation.py:65: UserWarning: The use of `File` for output is deprecated, please update your code to use `VTKFile` from `firedrake.output`.
  warn(


In [13]:
print(output_dir)

output
